In [1]:
from google.colab import drive

drive.mount('/content/gdrive/', force_remount=True)

Mounted at /content/gdrive/


In [2]:
import os
import json
import pandas as pd

def create_video_dataframe(video_dir, labels_file_path, exciting_threshold=0.7, unexciting_threshold=0.3, output_file_path='video_tags_test.csv'):
    # Load the json file containing the labels
    with open(labels_file_path, 'r') as f:
        labels = json.load(f)

    # Initialize an empty list to store the dataframe rows
    rows = []

    # Loop over the videos in the directory and extract their tags
    for video_name in os.listdir(video_dir):
        video_id = os.path.splitext(video_name)[0]  # Get video ID without extension
        label_score = labels.get(video_id, 0)  # Get label score from the json file

        if label_score > exciting_threshold:
            tag = 'exciting'
        elif label_score <= unexciting_threshold:
            tag = 'unexciting'
        else:
            tag = None

        if tag is not None:
            rows.append({'video_name': video_name, 'tag': tag})

    # Create a pandas dataframe from the rows list
    df = pd.DataFrame(rows)

    # Save the dataframe to a CSV file
    df.to_csv(output_file_path, index=False)

    return df


# df = create_video_dataframe('/Users/bunnylu/Desktop/FYP/database/video/',
# '/Users/bunnylu/Desktop/FYP/video_Exciting_clean.json', 0.5, 0.5)
df = create_video_dataframe('/content/gdrive/MyDrive/100video/',
                            '/content/gdrive/MyDrive/dataset/video_Exciting_clean.json', 0.5, 0.5).drop_duplicates()


In [4]:
df = pd.read_csv('/content/video_tags_test.csv')
tag_counts = df['tag'].value_counts()
print(tag_counts)

exciting      60
unexciting    40
Name: tag, dtype: int64


In [ ]:
import cv2
import numpy as np
import os
import pandas as pd
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical

def load_videos(video_folder, video_info, num_frames, height, width, num_classes):
    X = []
    y = []
    labels = {'exciting': 0, 'unexciting': 1}

    for index, row in video_info.iterrows():
        video_file = row['video_name']
        label = row['tag']

        if not video_file.endswith('.mp4'):
            continue

        video_path = os.path.join(video_folder, video_file)
        print(video_path)
        cap = cv2.VideoCapture(video_path)
        frames = []

        while True:
            ret, frame = cap.read()
            if not ret:
                break

            frame = cv2.resize(frame, (width, height))
            frames.append(frame)

            if len(frames) == num_frames:
                X.append(np.array(frames))
                y.append(labels[label])
                frames = []

        cap.release()

    return np.array(X), np.array(y)

video_folder = '/content/gdrive/MyDrive/100video/'
csv_path = '/content/video_tags_test.csv'

video_info = pd.read_csv(csv_path)

num_frames = 16
height = 112
width = 112
num_classes = 2

X, y = load_videos(video_folder, video_info, num_frames, height, width, num_classes)
X = X.astype('float32') / 255.0  # Normalize pixel values
y = to_categorical(y, num_classes)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [4]:
from keras.layers import Dense, Flatten, Dropout, ZeroPadding3D
from keras.layers import LSTM
from keras.models import Sequential, load_model
from keras.optimizers import Adam, RMSprop
from keras.layers import TimeDistributed
from keras.layers.convolutional import (Conv2D, MaxPooling3D, Conv3D,
    MaxPooling2D)
from collections import deque
import sys
input_shape = [20, 224, 224, 3]
def conv_3d():
        """
        Build a 3D convolutional network, based loosely on C3D.
            https://arxiv.org/pdf/1412.0767.pdf
        """
        # Model.
        model = Sequential()
        model.add(Conv3D(
            32, (3,3,3), activation='relu', input_shape=input_shape
        ))
        model.add(MaxPooling3D(pool_size=(1, 2, 2), strides=(1, 2, 2)))
        model.add(Conv3D(64, (3,3,3), activation='relu'))
        model.add(MaxPooling3D(pool_size=(1, 2, 2), strides=(1, 2, 2)))
        model.add(Conv3D(128, (3,3,3), activation='relu'))
        model.add(Conv3D(128, (3,3,3), activation='relu'))
        model.add(MaxPooling3D(pool_size=(1, 2, 2), strides=(1, 2, 2)))
        model.add(Conv3D(256, (2,2,2), activation='relu'))
        model.add(Conv3D(256, (2,2,2), activation='relu'))
        model.add(MaxPooling3D(pool_size=(1, 2, 2), strides=(1, 2, 2)))

        model.add(Flatten())
        model.add(Dense(1024))
        model.add(Dropout(0.5))
        model.add(Dense(1024))
        model.add(Dropout(0.5))
        model.add(Dense(2, activation='softmax'))

        return model

model = conv_3d()
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv3d_1 (Conv3D)           (None, 18, 222, 222, 32)  2624      
                                                                 
 max_pooling3d_1 (MaxPooling  (None, 18, 111, 111, 32)  0        
 3D)                                                             
                                                                 
 conv3d_2 (Conv3D)           (None, 16, 109, 109, 64)  55360     
                                                                 
 max_pooling3d_2 (MaxPooling  (None, 16, 54, 54, 64)   0         
 3D)                                                             
                                                                 
 conv3d_3 (Conv3D)           (None, 14, 52, 52, 128)   221312    
                                                                 
 conv3d_4 (Conv3D)           (None, 12, 50, 50, 128)  